In [ ]:
using Distributed
ifelse(nprocs()<2, addprocs(3), nothing);

In [ ]:
using Plots
using LinearAlgebra
using Revise
using LatticeQM

# Tutorial: Twisted honeycomb lattices

In [ ]:
lat = Geometries2D.honeycomb_twisted(8)
Structure.Lattices.foldPC!(lat; shift=[1/3,1/3,0])

In [ ]:
plot(lat, 3; supercell=Structure.Lattices.getneighborcells(lat, 1; halfspace=false, innerpoints=true, excludeorigin=false), sort="layer", markersize=3)

## Model

In [ ]:
valley = Operators.valley(lat; spinhalf=false)
hops = Operators.graphene(lat; tz=0.45, format=:sparse, mode=:nospin) # specifying format is important here!
Operators.setfilling!(hops,0.5; nk=12)
nothing

### Bands

In [ ]:
ks = kpath(lat, ["μ2", "γ", "κ", "κ'", "γ", "μ"]; num_points=100)
bands = getbands(hops, ks, valley; format=:sparse, num_bands=25)

In [ ]:
p1 = plot(bands; size=(300,250), markercolor=:diverging_bkr_55_10_c35_n256, colorbar=true)
p2 = plot(bands; ylim=(-0.0025,0.0025), size=(300,250), markercolor=:diverging_bkr_55_10_c35_n256, colorbar=true)

plot(p1,p2; size=(600,200))

In [ ]:
hops = Operators.graphene(lat; mode=:spinhalf, format=:sparse, tz=0.45)
Operators.addrashba!(hops, lat, r -> 0.04 * sign(r[3]))
Operators.addzeeman!(hops, lat, r -> 0.04 * [0.0, 0.0, sign(r[3])])
Operators.setfilling!(hops,0.5; nk=12)

# kgrid = LatticeQM.Utils.randomgrid(;nk=8^2, rot_symmetry=6, B=Structure.getB(lat))
kgrid = LatticeQM.Utils.regulargrid(;nk=8^2)
ωs = collect(range(-0.0015, length=100, stop=0.001))
ldos = Spectrum.ldos(hops, kgrid, ωs; Γ=0.0005, format=:sparse, num_bands=25);

In [ ]:
plot(lat, ldos; clims=(0,maximum(ldos)), markercolor=:inferno, colorbar=true)